# MTH 651: Advanced Numerical Analysis

## Homework Assignment 1

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
* The code must run in Colab without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Assignment Goals

* The purpose of this assignment is to develop a 1D finite element Poisson solver and analyze its accuracy in the energy and $L^2$ norms.

### Problem Statement

We want to solve the Poisson problem on $[0,1]$ with homogeneous Dirichlet boundary conditions
$$
   \begin{align*}
      -u''(x) &= f(x) \\
      u(0) &= 0 \\
      u(1) &= 1
   \end{align*}
$$

Recall the finite element formulation of the above problem:

> Find $u \in S$ such that, for all $v \in S$
> $$ a(u, v) = (f, v) $$
> where $S$ is the space of continuous piecewise linear functions defined in terms of a partition $0 = x_0 < x_1 < \cdots < x_n < x_{n+1} = 1$, and $a(\cdot,\cdot)$ is the bilinear form
> $$ a(u, v) = \int_0^1 u'(x) v'(x) \, dx $$
> and $(\cdot, \cdot)$ is the $L^2$ inner product,
> $$ (u, v) = \int_0^1 u(x) v(x) \, dx $$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Write a function that will create a mesh (partition) of the interval $[0,1]$ using $n$ points, including the boundaries, and return the points as an ordered numpy array.

For example, `make_uniform_mesh(5)` should return `array([0.  , 0.25, 0.5 , 0.75, 1.  ])`

In [8]:
def make_uniform_mesh(n):
    """
    Return a mesh of the interval [0,1] with a total of n points, including the 
    endpoints.
    """
    return

Recall that the finite element method will result in a linear system of equations
$$ \boldsymbol{K} \boldsymbol{U} = \boldsymbol{F} $$
where $\boldsymbol{K}$ is the stiffness matrix, $\boldsymbol{U}$ are the nodal values of the approximate solution $u_S$ (i.e. expansion coefficients in the nodal basis), and $\boldsymbol{F}$ is the right-hand side, i.e.
$$ F_i = (f, \phi_i) $$
for the "hat function" $\phi_i$.

Write a function that will **approximate** the right-hand side vector $\boldsymbol{F}$ using a **one-point quadrature formula**.

This will be an approximation, we won't be computing the integrals exactly. But it will make the code simpler.

The one-point quadrature approximation is
$$
   \int_a^b f(x) \,dx \approx (b-a) f\left( \tfrac{a+b}{2} \right)
$$

The function `make_rhs` should return a vector (numpy array) whose `i`th entry is the coefficient $F_i$ using this quadrature approximation.
The input will be the function `f` and the mesh points `x`.

Note that each basis function $\phi_i$ is nonzero in only **two intervals**, $[x_{i-1}, x_i]$ and $[x_i, x_{i+1}]$.
Therefore, the inner product $(f, \phi_i)$ can be written as
$$
   F_i = (f, \phi_i) = \int_0^1 f(x) \phi_i(x) \, dx = \int_{x_{i-1}}^{x_i} f(x)\phi_i(x) \,dx + \int_{x_i}^{x_{i+1}} f(x) \phi_i(x) \, dx.
$$
Approximate each of the two integrals on the right-hand side of the above expression, and sum them to approximate $F_i$.

The first and last mesh point (corresponding to $x = 0$ and $x = 1$) do not have associated basis functions because of the Dirichlet boundary conditions.
So, the first and last entry of the returned array can be set to zero.

In other words, the result should be
$$
   \texttt{make\_rhs}(f, x) = [0, F_1, F_2, \ldots, F_{n-2}, 0]
$$

In [ ]:
def make_rhs(f, x):
   """
   Return the right-hand side vector F given the function f and mesh points x.

   Set the first and last entries to zero, i.e.
   F[0] = 0
   F[-1] = 0
   """
   return

We now want to form the stiffness matrix $\boldsymbol{K}$.
This is the most challenging part of implementing a finite element method.

Recall that $K_{ij} = a(\phi_i, \phi_j) = (\phi_i', \phi_j')$.

As before, note that $\phi_i$ is nonzero only on two intervals, and similarly for $\phi_j$.
Therefore, $a(\phi_i, \phi_j)$ will be nonzero only if there is some overlap between these two sets of two intervals.
In particular,
$$
   a(\phi_i, \phi_j) \text{ is nonzero iff } j \in \{ i-1, i, i+1 \}
$$

The usual way to **assemble** the stiffness matrix is to loop over every element (each interval in the mesh), and to compute the contributions to each relevant entry $K_{ij}$.

Note that on each element, $\phi_i'$ is constant, and so the integral $\int_a^b \phi_i' \phi_j' \, dx$ can be computed _exactly_ by the one-point quadrature rule. (Verify this?)

Write a function `make_stiffness_matrix` that will return the stiffness matrix $\boldsymbol{K}$ given the mesh points `x` as input.
Because of the boundary conditions, we don't need to compute the entries corresponding to the first and last point ($x=0$ and $x=1$).
Set those rows and columns of the matrix equal to the identity (i.e. $K_{0,0} = 1$ and $K_{n-1,n-1} = 1$, $K_{0,i} = K_{i,0} = K_{0,n-1} = K_{n-1,0} = 0$).

In [9]:
def make_stiffness_matrix(x):
    """
    Return the assembled stiffness matrix K given the mesh points X.

    Set the first and last rows and columns to identity, i.e.

    K[0,:] = 0
    K[:,0] = 0
    K[0,0] = 1

    K[:,-1] = 0
    K[:,-1] = 0
    K[-1,-1] = 1

    Loop over each element, and integrate the products of the basis functions 
    that are nonzero on that element (on each interval, there are two nonzero
    basis functions corresponding to the interval endpoints). This leads to four
    pairs of basis functions to integrate. Add the integrals to the stiffness
    matrix and return the result. Be careful not to include basis functions 
    corresponding to 0 and 1.
    """
    return

Write a function to use the finite element method to solve the Poisson problem with Dirichlet BCs given a right-hand side function $f$ and mesh.
Return the result as a vector (numpy array) $\boldsymbol{U}$, whose entries are the nodal values $U_i$.

In [ ]:
def solve_poisson(f, x):
    """
    Return the finite element approximation U to the Poisson problem with
    right-hand side f and mesh x.

    You can use np.linalg.solve to solve the linear system of equations
    """
    return

Use the functions written above to solve the Poisson problem with right-hand side $f(x) = \sin(x)$.
Solve using a uniformly spaced mesh with 101 points.

Plot the result. Compute (by hand) the formula for the exact solution, and plot the exact solution to compare.

Write functions to approximate energy norm and $L^2$ norm errors.

Each function should take the mesh points `x`, a vector `u` of nodal values (interpreted as a piecewise linear finite element function), and the exact solution `u_exact` (in the case of $L^2$ error) or `du_exact` (representing the exact derivative $u'(x)$ in teh case of the energy norm).

Approximate the integrals $\int_0^1 (u - u_S)^2 \, dx$ and $\int_0^1 (u' - u_S')^2 \, ds$ using one-point quadrature.

In [ ]:
def energy_error(x, u, du_exact):
    return

def l2_error(x, u, u_exact):
    return

Perform a converge study.

Start with a uniformly spaced mesh with 10 elements (21 points).
Compute the finite element approximation, and the error when compared with the exact solution (in both energy and $L^2$ norms).
Double the number of elements, and recompute.
Repeat this process to obtain a total of six solutions on meshes of increasing resolution (10 elements through 320 elements).
Compute the empirical rates of convergence.
Do they agree with the theory? Why or why not?

### Questions

1. What is the order of accuracy of the one-point quadrature approximation (in terms of the interval length $h = b-a$)? Prove it. Hint: Taylor's theorem.
   * Also prove that the entries of $K_{ij}$ can be computed exactly using one-point quadrature.
2. Prove that $ \| u - u_I \| \leq C h^2 \| u'' \| $ (see exercise 0.x.6 from the textbook for a hint).